In [1]:
import sys
import numpy as np
filename = 'Dataset.csv'

nbLignes = 0

X_train=[]
Y_train=[]

for line in open(filename,"r"):
    ligne=line.split(',')
    text=''
    for i in range(3,len(ligne)):
        if (i!=len(ligne)-1):
            text=text+ligne[i]+","
        else:
            text+=ligne[i]
    X_train.append(text)
    Y_train.append(ligne[1])
    nbLignes=nbLignes+1
    
X_train=np.asarray(X_train)
Y_train=np.asarray(Y_train)

X_test=X_train[-300000:]
Y_test=Y_train[-300000:]

X_train=X_train[1:300001]
Y_train=Y_train[1:300001]

In [4]:
print X_train[0]

                     is so sad for my APL friend.............



In [5]:
def get_word(x):
    word=""
    word_list=[]
    flag=True
    for i in range(len(x)):
        if x[i].isalpha() or x[i]=="@" or x[i]=="&":
            if flag:
                word+=x[i]
                if i+1==len(x):
                    word_list.append(word)
            else:
                flag=True
                word+=x[i]

        else:
            flag=False
            if word!="":
                word_list.append(word)
            word=""
        if word=="http" or word=="www":
            word_list.append(x)
            break
    return word_list

In [6]:
dictionary={}
dictionary_inverse={}
couts=[]
k=0
for sentense in X_train:
    word_list=get_word(sentense)
    for word in word_list:
        if word not in dictionary:
            dictionary[word]=k
            k+=1
            dictionary_inverse[k]=word
            couts.append(1)
        else:
            index=dictionary[word]
            couts[index]+=1

In [7]:
new_dictionary={}
new_dictionary_inverse={}
k=1
for i in range(len(couts)):
    if (couts[i]>20):
        if (couts[i]<10000):
            word=dictionary_inverse[i]
            new_dictionary[word]=k
            new_dictionary_inverse[k]=word
            k+=1

In [8]:
def change_into_number(sentense):
    word_list=get_word(sentense)
    l=[]
    for i in range(len(word_list)):
        if word_list[i] in new_dictionary:
            k=new_dictionary[word_list[i]]
            if k not in l:
                l.append(k)
        else:
            l.append(0)
    return l

In [9]:
New_X_train=[]
New_X_test=[]
for tweet in X_train:
    New_X_train.append(change_into_number(tweet))
train=np.asarray(New_X_train)

for sentense in X_test:
    New_X_test.append(change_into_number(sentense))
test=np.asarray(New_X_test)

In [12]:
print train[0]
print X_train[0]

[0, 1, 0, 0, 0, 2, 0]
                     is so sad for my APL friend.............



In [26]:
def nb_y_one(data_set):
    k=0

    for i in data_set:
        if i == 1:
            k+=1

    return k


In [18]:
Y_train=np.asarray([int(y) for y in Y_train])
Y_test=np.asarray([int (y ) for y in Y_test])

beacause the data in Y isn't the type of int,
so we need to change the form of the Y 

In [9]:
train_Py=[]
test_Py=[]

nb_train_y1=nb_y_one(Y_train)
train_Py.append((len(Y_train)-nb_train_y1)/float(len(Y_train)))
train_Py.append(nb_train_y1/float(len(Y_train)))

nb_test_y1=nb_y_one(Y_test)
test_Py.append((len(Y_test)-nb_test_y1)/float(len(Y_test)))
test_Py.append(nb_test_y1/float(len(Y_test)))

the function nb_y_one aimed to calucale the nb of the jugement which is 1(positive)

Also,the train_Py has 2 dimension,train_Py[0] stock the nb of jugement 0
train_Py[1] stock the nb of the jugement 1

The same for the test_Py

In [10]:
def word_jug(data_set,label):
    positive=[0 for i in range(len(new_dictionary)+1)]
    negatif=[0 for i in range(len(new_dictionary)+1)]
    for i in range(len(data_set)):
        sentence=data_set[i]
        for index in sentence:
            if label[i]==1:
                positive[index]+=1
            else:
                negatif[index]+=1
    return negatif,positive

the function word_jug is for calucle how many times the words show in the negatif or positive sentence

In [11]:

l1,l2=word_jug(train,Y_train)

In [12]:
train_word_juge=[]
train_word_juge.append(l1)
train_word_juge.append(l2)

train_word_juge has 2 dimension
train_word_juge[0] stock how many times the word show in the sentence negative 
train_word_juge[1] stock how many times the word show in the sentence positive

In [13]:
def proba_juge(juge,nby1,nby0):
    p0=[]
    p1=[]
    for i in range(len(juge[1])):
        p1.append((juge[1][i]+1)/float((nby1+10000)))
        p0.append((juge[0][i]+1)/float((nby0+10000)))
    return p0,p1
        
        

p1=P(xi|y=1)   p0=P(xi|y=0)
and for the laplace smooth, what i do is add 1 on the numerator, add 10000(the size of the vocabulary) on the denominator

In [14]:

train_p=[]
p0,p1=proba_juge(train_word_juge,nb_train_y1,(len(Y_train)-nb_train_y1))
train_p.append(p0)
train_p.append(p1)

train_p[0]=P(xi|y=0)
trian_p[1]=P(xi|y=1)

Because our goal is to compare the pro of P(z=1|xi) and P(z=0|xi)
we have the formula:
    P(z=1|xi)=P(xi|z=1)*P(z=1)/the same things
    So 
        if P(xi|z=1)*P(z=1)>P(xi|z=0)*P(z=0)
            then the predire label is 1
        else
            label is 0

In [15]:
def predict_class(test):
    standard_test=test
#    for tweet in test:
#        standard_test.append(change_into_number(tweet))
    standard_test=np.asarray(standard_test)
    p1=1
    p0=1
    label=[]
    for tweet in standard_test:
        for index in tweet:
            p1*=train_p[1][index]
            p0*=train_p[0][index]
        p1*=train_Py[1]
        p0*=train_Py[0]
        if p1>p0:
            label.append(1)
        else:
            label.append(0)
        p1=1
        p0=1
    return label

        

In [18]:
Test=np.asarray(test)
label=predict_class(Test)

label=np.asarray(label)
mis=0
for i in range(len(label)):
    if Y_test[i]!=label[i]:
        mis+=1
print "the loss: ",mis/float(len(label))
print "the acc: ",(len(label)-mis)/float(len(label))

the loss:  0.30822
the acc:  0.69178


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True, max_features = 4098)
ctrain= cv.fit_transform(X_train)
ctest= cv.transform(X_test)



In [4]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB(alpha=0.5)
print type(Y_test[0])

<type 'numpy.string_'>


In [5]:
bnb.fit(ctrain,Y_train)

BernoulliNB(alpha=0.5, binarize=0.0, class_prior=None, fit_prior=True)

In [6]:
bnb.score(ctest,Y_test)

0.74821666666666664

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=False, max_features = 4098)
ctrain= cv.fit_transform(X_train)
ctest= cv.transform(X_test)


from sklearn.naive_bayes import MultinomialNB
bnb = MultinomialNB(alpha=0.5)
bnb.fit(ctrain,Y_train)
bnb.score(ctest,Y_test)

0.75736666666666663